In [1]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
import csv
import bar_chart_race as bcr
import matplotlib as plt 

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None) # to see the complete DF and not the truncated version

In [3]:
league_url ='https://fantasy.premierleague.com/api/leagues-classic/1771282/standings/'
r = requests.get(league_url).text
json_data = json.loads(r)
# print (json.dumps(json_data, indent=2))

In [4]:
#converting to DF

In [5]:
league_data = pd.DataFrame(json_data['standings']['results'])

In [6]:
#creating list of dataframes for each individual player 

In [24]:
all_dataframes = []
player_id = league_data['entry'].tolist()
all_name = iter(league_data["player_name"].tolist())
result = []
no_gw = 9 # total no. of GW for the league

for team in player_id:

    team_url = 'https://fantasy.premierleague.com/api/entry/' + str(team) + '/history/' #inidividual address for each player
    p_id = requests.get(team_url).text
    teams_json_data = json.loads(p_id)
#     print(json.dumps(teams_json_data, indent =2))  #prints JSON for every player

    dataframe = pd.DataFrame(teams_json_data['current'], columns = ["total_points"])  #create dataframes for each player       
#     dataframe = pd.DataFrame(row, columns = ["GW", "total_points"])

    # accounting for old accounts 
    if len(dataframe['total_points']) >= 20:
        dataframe = dataframe.diff()
        dataframe = dataframe.iloc[-6:]
        dataframe = dataframe.cumsum()
        
        
                
    dataframe = dataframe.rename(columns = {"total_points": next(all_name)})
    dataframe = dataframe.reset_index(drop=True)
    all_dataframes.append(dataframe)

In [25]:
final_dfs = pd.concat(all_dataframes, axis=1)

In [26]:
# Add GW columns here
GW = ["30+","31+","32+", "33+","34+","35+"]
final_dfs.insert (0, "GW", GW)

In [27]:
final_dfs = final_dfs.set_index("GW")

In [28]:
final_dfs

,Mahbube Rabbani,shareque zaman,Mubtasim Fuad,Saraf Hasan,Tasnimul Hasan,Pial Rahman,Tauhid Sefo,Likhon Amin,Mahedur Rahman,Abrar Hossain,Muntasir Azad,Asif Bin Zaman,Zabir Sami
GW,,,,,,,,,,,,,
30+,61,95,69,72,59,92,66.0,64.0,77.0,56,55.0,61,73
31+,124,164,137,135,114,148,140.0,113.0,151.0,125,121.0,113,148
32+,187,260,230,220,199,209,196.0,190.0,204.0,223,176.0,162,184
33+,259,311,286,288,259,259,267.0,245.0,246.0,277,233.0,228,240
34+,357,371,347,346,343,329,336.0,318.0,304.0,327,289.0,302,291
35+,409,403,400,397,395,394,378.0,372.0,359.0,358,351.0,347,334


In [12]:
# Visuualization

In [40]:

bcr.bar_chart_race(
    df=final_dfs,
    filename= 'Nuno Nothing Jon Snow Dancing Chair.mp4',
    orientation='h',
    sort='desc',
    n_bars=13,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=85,
    interpolate_period=False,
    label_bars=True,
    bar_size=.95,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='GW {x}',
    period_summary_func=lambda v, r: {'x': .99, 'y': .38,
                                      's': f'Highest point in GW: {v.nlargest(13).max():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},
    perpendicular_bar_func='median',
    period_length=6500,
    figsize=(5, 3),
    dpi=144,
    cmap='dark12',
    title='Nuno Nothing Jon Snow Dancing Chair',
    title_size='',
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=False)  